## Question 3

**Implications**

1) Dominance of Market Returns 
- if you're shorting SPY as a hedge, then even when other large caps are underperforming, the SP500 could still be going up due to MAG7.

2) Higher Volatility of Large-Cap Basket
- Implication: The tech-heavy nature of the Magnificent 7 introduces increased volatility in the large-cap space. Shorting the entire large-cap index may expose the strategy to elevated risk driven by sudden rallies or corrections in these concentrated stocks.

3) Beta Instability
- Due to the incresed vol in the index, your hedge ratio could be off. 

4) Value vs Growth 
Tech stocks are growth, while small caps are usually value. This strategy could be picking up undo risks due to that. Hedging them to make the strategy more concentrated would be a better idea. 

In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
import plotly.graph_objects as go

**Baseline Strategy**

We assume that the baseline strategy for the long-small cap/short large cap is long SP600 and short SPY, equally weighted. We'll then show how this can be improved by constructing the strategy on a sector neutral basis.

In [71]:
def backtest_pair(small,large,start,end):

    df = yf.download(tickers = [small,large], start = start, end = end)['Close']
    df_ret = df.pct_change().dropna()
    df_port_ret = df_ret[small] - df_ret[large]
    df_port_ret = pd.DataFrame(df_port_ret,columns = ['daily_return'])
    df_port_ret['cumulative_return'] = df_port_ret['daily_return'].cumsum()
    return df_port_ret

In [72]:
base = backtest_pair('^SP600','SPY',start = date(2020,1,1),end = date(2025,1,1))
base

[                       0%                       ]

[*********************100%***********************]  2 of 2 completed


,daily_return,cumulative_return
Date,,
2020-01-03,0.007670,0.007670
2020-01-06,-0.004961,0.002709
2020-01-07,-0.002749,-0.000040
2020-01-08,-0.002943,-0.002982
2020-01-09,-0.007489,-0.010472
...,...,...
2024-12-24,-0.001843,-0.272903
2024-12-26,0.005213,-0.267690
2024-12-27,-0.002819,-0.270509


In [73]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def backtest_pair_hedge(small, large, start, end, market='^GSPC'):
    # Download price data
    df = yf.download(tickers=[small, large, market], start=start, end=end)['Close']
    
    # Calculate daily returns
    df_ret = df.pct_change().dropna()
    
    # Portfolio returns (small-cap long, large-cap short)
    df_port_ret = df_ret[small] - df_ret[large]
    df_port_ret = pd.DataFrame(df_port_ret, columns=['daily_return'])
    
    # Include market returns for beta estimation
    df_port_ret['market_return'] = df_ret[market]
    
    # Estimate beta using linear regression
    X = df_port_ret[['market_return']].values.reshape(-1, 1)
    y = df_port_ret['daily_return'].values
    reg = LinearRegression().fit(X, y)
    beta = reg.coef_[0]
    
    print(beta)
    # Adjust portfolio returns for market beta hedging
    df_port_ret['hedged_return'] = df_port_ret['daily_return'] - beta * df_port_ret['market_return']
    
    # Compute cumulative returns
    df_port_ret['cumulative_return'] = df_port_ret['hedged_return'].cumsum()
    
    return df_port_ret


In [76]:
sector_etfs = {
    'Baseline': {
        "Large-Cap": "SPY",
        "Small-Cap": "^SP600"
    },
    "Technology": {
        "Large-Cap": "XLK",
        "Small-Cap": "PSCT"
    },
    "Healthcare": {
        "Large-Cap": "XLV",
        "Small-Cap": "PSCH"
    },
    "Financials": {
        "Large-Cap": "XLF",
        "Small-Cap": "PSCF"
    },
    "Consumer Discretionary": {
        "Large-Cap": "XLY",
        "Small-Cap": "PSCD"
    },
    "Consumer Staples": {
        "Large-Cap": "XLP",
        "Small-Cap": "PSCC"
    },
    "Industrials": {
        "Large-Cap": "XLI",
        "Small-Cap": "PSCI"
    },
    "Energy": {
        "Large-Cap": "XLE",
        "Small-Cap": "PSCE"
    },
    "Materials": {
        "Large-Cap": "XLB",
        "Small-Cap": "PSCM"
    },
    "Utilities": {
        "Large-Cap": "XLU",
        "Small-Cap": "PSCU"
    },
    "Real Estate": {
        "Large-Cap": "XLRE",
        "Small-Cap": "ROOF"
    }
}


fig = go.Figure()

for sector,(large,small) in sector_etfs.items():
    large_cap_ticker = sector_etfs[sector][large]
    small_cap_ticker = sector_etfs[sector][small]
    # print(f'Sector: {sector}, Large Cap: {large_cap_ticker}, Small Cap {small_cap_ticker}')
    sector_long_short = backtest_pair_hedge(small_cap_ticker,large_cap_ticker, start = date(2010,1,1),end = date(2025,1,1))

    fig.add_trace(
        go.Scatter(
            x = sector_long_short.index,
            y = sector_long_short['cumulative_return'],
            name = sector,
            opacity= 0.5 if sector != 'Baseline' else 1
        )
    )

fig.update_layout(title = 'Sector Long/Short (Equal Weight, No Hedging)')
fig.update_yaxes(title = 'Cumulative Return')
fig.show()
    

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed


0.1310262863897647
0.02596615283885883


[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed


0.21638286740794782
-0.1056302108121618


[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed


...

0.06750996673163014
0.2813838519118692


[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed


-0.050850983915759936
0.09870785084969572


[*********************100%***********************]  3 of 3 completed
/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_73961/2640344886.py:11: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



0.08038140275354669


In [75]:
tech_no_hedge = backtest_pair('PSCT','XLK',start = date(2015,1,1),end = date(2019,1,1))
tech_w_hedge = backtest_pair_hedge('PSCT','XLK',start = date(2015,1,1),end = date(2019,1,1))

fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = tech_no_hedge.index,
        y = tech_no_hedge['cumulative_return'],
        name = 'No Hedge'
    )
)

fig.add_trace(
    go.Scatter(
        x = tech_w_hedge.index,
        y = tech_w_hedge['cumulative_return'],
        name = 'Hedged'
    )
)


fig.update_layout(title = 'Tech, Equal Weight (L/S) + Market Neutral Hedge')
fig.show()

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  3 of 3 completed


-0.0867351207235465


In [105]:
import yfinance as yf
import pandas as pd
import numpy as np

def hedge_factors(small, large, start, end, factor_tickers, window=60):
    """
    Backtest a strategy that goes long on small-cap ETF and short on large-cap ETF with factor hedging.
    
    Parameters:
    small (str): Ticker for the small-cap ETF
    large (str): Ticker for the large-cap ETF
    start (str): Start date for the data (e.g., '2010-01-01')
    end (str): End date for the data (e.g., '2025-01-01')
    factor_tickers (list): List of factor ETF tickers (e.g., market, value, growth, momentum)
    window (int): Rolling window size for factor exposures (default is 60 days)
    
    Returns:
    pd.Series: The factor-hedged return series of the strategy
    """
    # Download adjusted close price data for the tickers
    tickers = [small, large] + factor_tickers  # Include factor ETFs
    data = yf.download(tickers, start=start, end=end)['Close']
    
    # Calculate daily returns for all tickers
    returns = data.pct_change().dropna()
    
    # Compute portfolio returns for the small-cap long, large-cap short strategy
    portfolio_returns = returns[small] - returns[large]
    
    # Hedging logic using a rolling window
    hedged_returns = []
    
    for i in range(window, len(returns)):
        # Define the rolling window of data
        window_returns = returns.iloc[i-window:i]
        window_portfolio = portfolio_returns.iloc[i-window:i]
        
        # Hedge against each factor using linear regression
        factor_exposures = {}
        
        for factor in factor_tickers:
            X_factor = window_returns[factor].values.reshape(-1, 1)
            y_portfolio = window_portfolio.values
            factor_exposure = np.linalg.lstsq(X_factor, y_portfolio, rcond=None)[0]
            factor_exposures[factor] = factor_exposure[0]
        
        # Apply the factor exposures to remove their influence (adjust returns)
        portfolio_hedged = window_portfolio.copy()  # Make a copy of the portfolio returns to adjust
        
        for factor in factor_tickers:
            portfolio_hedged -= factor_exposures[factor] * window_returns[factor]
        
        # Append the final hedged return for this period
        hedged_returns.append(portfolio_hedged.iloc[-1])
    
    # Convert the hedged returns into a pandas series
    hedged_returns_series = pd.Series(hedged_returns, index=portfolio_returns.index[window:])
    
    return hedged_returns_series

# Example usage:
small_cap_tech_ticker = 'PSCF'  # Invesco Small Cap Tech ETF
large_cap_tech_ticker = 'XLF'   # Large Cap Tech ETF
start_date = '2010-01-01'
end_date = '2025-01-01'

# Define the factor ETFs for hedging (market, value, growth, momentum)
factor_tickers = ['SPY', 'VTV', 'VUG', 'MTUM']

# Perform backtest and factor hedging
hedged_returns = hedge_factors(small_cap_tech_ticker, large_cap_tech_ticker, start_date, end_date, factor_tickers, window=30)

print(hedged_returns)

[*********************100%***********************]  6 of 6 completed


Date
2013-06-03   -0.003415
2013-06-04    0.004942
2013-06-05    0.001653
2013-06-06   -0.000017
2013-06-07   -0.006800
                ...   
2024-12-24   -0.010932
2024-12-26   -0.018801
2024-12-27    0.001771
2024-12-30    0.008394
2024-12-31    0.020292
Length: 2916, dtype: float64


In [106]:
returns = pd.DataFrame(hedged_returns,columns=['daily_return'])
returns['cumulative_return'] = (1+returns['daily_return']).cumprod()
returns

,daily_return,cumulative_return
Date,,
2013-06-03,-0.003415,0.996585
2013-06-04,0.004942,1.001510
2013-06-05,0.001653,1.003165
2013-06-06,-0.000017,1.003148
2013-06-07,-0.006800,0.996327
...,...,...
2024-12-24,-0.010932,0.625416
2024-12-26,-0.018801,0.613657
2024-12-27,0.001771,0.614744


In [110]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = returns.index,
        y = returns['cumulative_return'],
        name = 'Financials',
        showlegend=True
    )
)

fig.update_layout(title = 'Sector L/S w Factor Hedging (Value,Growth,Momentum,Market)')
fig.show()